In [1]:
import numpy as np
import pandas as pd
data = pd.read_csv('assignment4a.csv')
data.date = pd.to_datetime(data.date)
secs = ['Exxon Mobil', 'Procter & Gamble', 'Pfizer', 'Walmart ', 'Intel']
rf = '30 Day T-Bill'

r_premium = data[secs]
for sec in secs:
    r_premium[sec] -= data[rf]

## 1-A


In [2]:
# r_hat
r_hat = pd.Series(index = secs)
for sec in secs:
    r_hat[sec] = r_premium[sec].mean()
# cov_hat
cov_hat = pd.DataFrame(columns = secs, index = secs)
for sec in secs:
    for sec2 in secs:
        cov_hat[sec][sec2] = data[sec].cov(data[sec2])

In [3]:
cov_inv = np.mat(cov_hat.values,dtype = 'float').I
w = cov_inv@r_hat.values.reshape((5,1))
w = w/w.sum()
print('tangency portfolio is')
print(w)

tangency portfolio is
[[0.35981595]
 [0.11546475]
 [0.10449339]
 [0.28022511]
 [0.1400008 ]]


In [4]:
r_hat_round = round(r_hat*100)/100
w_us_round = cov_inv@r_hat_round.values.reshape((5,1))
w_round = w_us_round/w_us_round.sum()
print('tangency portfolio after rounding r is')
print(w_round)

tangency portfolio after rounding r is
[[0.42794154]
 [0.3010659 ]
 [0.08517738]
 [0.04732312]
 [0.13849206]]


In [5]:
w_mv = cov_inv.sum(axis = 1)
w_mv = w_mv/w_mv.sum()
print('minimum variance portfolio is')
print(w_mv)

minimum variance portfolio is
[[ 0.48408506]
 [ 0.31172185]
 [ 0.12538609]
 [ 0.08811868]
 [-0.00931167]]


## 1_B

In [6]:
w_im = np.identity(5)@r_hat.values.reshape((5,1))
w_im = w_im/w_im.sum()
print('tangency portfolio with identity variance is')
print(w_im)
w_im_round = np.identity(5)@r_hat_round.values.reshape((5,1))
w_im_round = w_im_round/w_im_round.sum()
print('tangency portfolio with identity variance after rounding r is')
print(w_im_round)

tangency portfolio with identity variance is
[[0.1325108 ]
 [0.12528056]
 [0.15007856]
 [0.27130118]
 [0.3208289 ]]
tangency portfolio with identity variance after rounding r is
[[0.16666667]
 [0.16666667]
 [0.16666667]
 [0.16666667]
 [0.33333333]]



## 1-C

In [7]:
r_m = 0.005
beta = np.array([0.6,0.7,1.2,0.9,1.2])
r_CAPM = pd.Series(data = beta * r_m, index = secs)
w_CAPM = cov_inv@r_CAPM.values.reshape((5,1))
w_CAPM = w_CAPM/w_CAPM.sum()
print('tangency portfolio using CAPM is')
print(w_CAPM)

tangency portfolio using CAPM is
[[0.23562901]
 [0.18876788]
 [0.39993247]
 [0.11516405]
 [0.06050659]]


## 1-D

In [24]:
r_bayesian = (r_CAPM + r_hat)/2
sigma = np.mat(cov_hat.values).diagonal().mean()
cov_bayesian = (cov_hat + sigma * np.identity(5))/2
cov_bayesian_inv = np.mat(cov_bayesian.values,dtype = 'float').I
w_baysian = cov_bayesian_inv@r_bayesian.values.reshape((5,1))
w_baysian = w_baysian/w_baysian.sum()
print('tangency portfolio using Bayesian is')
print(w_baysian)

tangency portfolio using Bayesian is
[[0.18874911]
 [0.14995926]
 [0.20797496]
 [0.24974542]
 [0.20357126]]


In [121]:
A_return = pd.Series()
B_return = pd.Series()
C_return = pd.Series()
D_return = pd.Series()

for i in range(5,47): #(1,47)
    data_available = data.iloc[0:12*i,]
    r_p = r_premium.iloc[0:12*i,]
    r_p_next = r_premium.iloc[12*i:12*(i+1),]
    # A: tangency portfolio using estimated r + estimated covariance
    r_hat = pd.Series(index = secs)
    for sec in secs:
        r_hat[sec] = r_p[sec].mean()
    cov_hat = pd.DataFrame(columns = secs, index = secs)
    for sec in secs:
        for sec2 in secs:
            cov_hat[sec][sec2] = data_available[sec].cov(data_available[sec2])
    cov_inv = np.mat(cov_hat.values,dtype = 'float').I
    w = cov_inv@r_hat.values.reshape((5,1))
    w = w/w.sum()
    A_return = pd.concat([A_return,(w.A.reshape(5) * r_p_next).sum(axis = 1)])

    # B: tangency portfolio with estimated r + identity variance
    w_im = np.identity(5)@r_hat.values.reshape((5,1))
    w_im = w_im/w_im.sum()
    B_return = pd.concat([B_return,(w_im.reshape(5) * r_p_next).sum(axis = 1)])

    # C: tangency portfolio with CAPM r + estimated variance
    w_CAPM = cov_inv@r_CAPM.values.reshape((5,1))
    w_CAPM = w_CAPM/w_CAPM.sum()
    C_return = pd.concat([C_return,(w_CAPM.A.reshape(5) * r_p_next).sum(axis = 1)])

    # D: tangency portfolio with Baysian r + variance
    r_bayesian = (r_CAPM + r_hat)/2
    sigma = np.mat(cov_hat.values).diagonal().mean()
    cov_bayesian = (cov_hat + sigma * np.identity(5))/2
    cov_bayesian_inv = np.mat(cov_bayesian.values,dtype = 'float').I
    w_bayesian = cov_bayesian_inv@r_bayesian.values.reshape((5,1))
    w_bayesian = w_bayesian/w_bayesian.sum()
    D_return = pd.concat([D_return,(w_bayesian.A.reshape(5) * r_p_next).sum(axis = 1)])
    


In [124]:
print(A_return.mean())
print(B_return.mean())
print(C_return.mean())
print(D_return.mean())

0.004643543972085544
0.013298864775883493
0.009232666775147245
0.013328509257871995


In [123]:
print(A_return.mean()/ A_return.std())
print(B_return.mean()/ B_return.std())
print(C_return.mean()/ C_return.std())
print(D_return.mean()/ D_return.std())

0.01723899711256686
0.22553979460391807
0.21835788218779653
0.2269321249279514
